In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, HeatMap

# ── 1) 데이터 읽기 ──────────────────────────────────────────
FILE = r'E:\Traffic-Safety\위치 데이터\data\불법주정차 신고현황(23년11월1일_24년3월13일).csv'
df = pd.read_csv(FILE)

# ── 2) 위·경도 열 이름 자동 찾기 & 정리 ───────────────────
#   - 한글/영문/대소문자 혼합 모두 대응
lat_col = next(c for c in df.columns if '위도' in c or c.lower().startswith('lat'))
lon_col = next(c for c in df.columns if '경도' in c or c.lower().startswith('lon'))

df[lat_col] = pd.to_numeric(df[lat_col], errors='coerce')
df[lon_col] = pd.to_numeric(df[lon_col], errors='coerce')
df = df.dropna(subset=[lat_col, lon_col])            # 결측 좌표 제거

# ── 3) 지도의 중심 계산 ────────────────────────────────────
center = [df[lat_col].mean(), df[lon_col].mean()]

# ── 4‑1) 포인트(클러스터) 지도 ─────────────────────────────
m_points = folium.Map(location=center, zoom_start=12, tiles='cartodbpositron')
cluster = MarkerCluster().add_to(m_points)

for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row[lat_col], row[lon_col]],
        radius=3,
        fill=True,
        fill_opacity=0.7,
        color='#3186cc',
        weight=0,
        popup=f"{lat_col}: {row[lat_col]:.5f}, {lon_col}: {row[lon_col]:.5f}"
    ).add_to(cluster)

m_points.save('illegal_parking_points.html')
print("✅ illegal_parking_points.html 생성 완료")

# ── 4‑2) 밀도(HeatMap) 지도 ───────────────────────────────
m_heat = folium.Map(location=center, zoom_start=12, tiles='cartodbpositron')
heat_data = df[[lat_col, lon_col]].values.tolist()
HeatMap(
    heat_data,
    radius=12,         # 픽셀 반경 (가까운 점을 어떻게 뭉칠지)
    max_zoom=13,       # 확대 시 사라지는 레벨
    blur=15,           # 퍼지는 정도
).add_to(m_heat)

m_heat.save('illegal_parking_heat.html')
print("✅ illegal_parking_heat.html 생성 완료")


✅ illegal_parking_points.html 생성 완료
✅ illegal_parking_heat.html 생성 완료


In [5]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# CCTV 데이터 불러오기
file = r'E:\Traffic-Safety\위치 데이터\data\불법주정차 신고현황(23년11월1일_24년3월13일).csv'
df = pd.read_csv(file)  # 파일 경로 맞게 조정

# 위도/경도 컬럼명 맞추기
df = df[['위도', '경도']].dropna()

# 서울 중심 좌표로 지도 생성
seoul_center = [37.5665, 126.9780]
m = folium.Map(location=seoul_center, zoom_start=11)

# 마커 클러스터
marker_cluster = MarkerCluster().add_to(m)

# CCTV 마커 추가
for _, row in df.iterrows():
    folium.Marker(location=[row['위도'], row['경도']]).add_to(marker_cluster)

# 지도 저장
m.save("seoul_cctv_map.html")

In [7]:
!pip install geopandas


[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 323.6/323.6 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 6.1 MB/s eta 0:00:00
     ---------------------------------------- 6.1/6.1 MB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 16.2/16.2 MB 8.1 MB/s eta 0:00:00


In [9]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point

# 1. CSV 파일 읽기 (경로 및 파일명을 실제 환경에 맞게 수정)
csv_file = r'E:\Traffic-Safety\위치 데이터\data\불법주정차 신고현황(23년11월1일_24년3월13일).csv'
df = pd.read_csv(csv_file)

# 2. 위도, 경도 컬럼만 선택하고 결측치 제거
df = df[['위도', '경도']].dropna()

# 3. DataFrame을 GeoDataFrame으로 변환
#    shapely의 Point 객체를 이용해서 각 행에 대한 geometry 생성 (경도, 위도 순)
geometry = [Point(xy) for xy in zip(df['경도'], df['위도'])]
gdf_points = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

# 4. 서울 구(자치구) 경계 GeoJSON 파일 읽기
#    실제 GeoJSON 파일 경로와 파일명을 환경에 맞게 수정하세요.
gu_geojson = r'E:\Traffic-Safety\위치 데이터\서울특별시.json'
gdf_gu = gpd.read_file(gu_geojson)

# 5. 공간 조인 (spatial join)을 통해 각 신고 데이터가 어느 구에 속하는지 판단
#    최신 geopandas 버전에서는 predicate 인자를 사용합니다.
#    여기서는 각 포인트가 구(polyon) 내부에 있는지를 확인합니다.
gdf_joined = gpd.sjoin(gdf_points, gdf_gu, how="left", predicate="within")

# 6. 공간 조인 후, GeoJSON 파일 속성 중 구 이름이 담긴 컬럼(예시에서는 'gu')를 기준으로 신고 건수 집계
#    만약 GeoJSON 파일 내 구 이름 컬럼명이 다르다면 아래 'gu' 부분을 수정하세요.
counts = gdf_joined.groupby('gu').size().reset_index(name='신고건수')

# counts DataFrame 예시 출력
print(counts.head())

# 7. Folium 지도 생성 (서울 중심 좌표 기준)
seoul_center = [37.5665, 126.9780]
m = folium.Map(location=seoul_center, zoom_start=11)

# 8. Choropleth 지도 생성
#    geo_data에는 GeoJSON 파일 경로(gu_geojson)를 그대로 사용합니다.
#    data로는 위에서 집계한 counts DataFrame을 사용합니다.
#    columns 파라미터에선 ['구이름컬럼', '신고건수']를 지정하며,
#    key_on 파라미터로 GeoJSON 내부 속성과 매칭(예: feature.properties.gu)합니다.
folium.Choropleth(
    geo_data=gu_geojson,
    name="choropleth",
    data=counts,
    columns=['gu', '신고건수'],
    key_on='feature.properties.gu',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='불법주정차 신고 건수'
).add_to(m)

# (선택) GeoJson Tooltip 추가: 각 구 영역에 마우스 올릴 때, 구 이름 및 건수 확인
folium.GeoJson(
    gu_geojson,
    name="구 정보",
    tooltip=folium.GeoJsonTooltip(
        fields=['gu'],  # GeoJSON 내 구 이름 필드 (필요시 다른 필드와 매칭)
        aliases=['구: '],
        localize=True
    )
).add_to(m)

# 9. 결과 지도를 HTML 파일로 저장
m.save("seoul_gu_choropleth.html")


KeyError: 'gu'

In [15]:
import json

# 미국 주별 실업률 데이터 불러오기
seoul = r'E:\Traffic-Safety\위치 데이터\서울특별시.json'
car_park = r'E:\Traffic-Safety\위치 데이터\data\불법주정차 신고현황(23년11월1일_24년3월13일).csv'
state_data = pd.read_csv(car_park)

m = folium.Map( [37.5665, 126.9780], zoom_start=3)

folium.Choropleth(
    geo_data=seoul, # 경계선 좌표값이 담긴 데이터
    data=state_data, # Series or DataFrame 넣으면 된다
    columns=['위도', '경도'],# 데이터프레임의 컬럼 지정
    key_on='feature.properties.adm_cd',  # GeoJSON 파일 내 'adm_cd' 필드와 매핑
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='서울특별시 값'
).add_to(m)
m.save("seoul_gu_choropleth.html")

In [ ]:
df = pd.read_csv(car_park)  # car_park는 csv 경로
df = df.dropna(subset=["위도", "경도"])

# 2. 위경도로 Point geometry 생성
geometry = [Point(xy) for xy in zip(df["경도"], df["위도"])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# 3. 서울시 행정동 GeoJSON 불러오기
seoul_gjson_path = r"E:\Traffic-Safety\위치 데이터\서울특별시.json"
seoul_gdf = gpd.read_file(seoul_gjson_path)

# 💡 '동'으로 끝나는 이름만 필터링
seoul_dong_gdf = seoul_gdf[seoul_gdf['name'].str.endswith('동')].copy()

# 4. 좌표 기반 Spatial Join
joined = gpd.sjoin(gdf, seoul_dong_gdf, how="inner", predicate='within')

# 5. '동' 기준 count 집계
joined_grouped = joined.groupby('name').size().reset_index(name='count')

# 6. Choropleth 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11, max_bounds=True)

folium.Choropleth(
    geo_data=seoul_dong_gdf,             # 🔸 동만 포함된 GeoDataFrame
    data=joined_grouped,                 # 🔸 동별 집계된 데이터
    columns=['name', 'count'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='서울시 행정동별 데이터 밀도',
).add_to(m)

m.save("seoul_dong_density_map.html")

,code,name,name_eng,base_year,geometry
0,3902062,예래동,Yeraedong,2013,"POLYGON ((126.45076 33.35525, 126.43109 33.314..."
1,3902061,중문동,Jungmun-dong,2013,"POLYGON ((126.50731 33.35441, 126.47668 33.309..."
2,3902060,대천동,Daecheon-dong,2013,"POLYGON ((126.52 33.35702, 126.5093 33.28029, ..."
3,3902059,대륜동,Daeryundong,2013,"POLYGON ((126.55147 33.24541, 126.54568 33.234..."
4,3902058,서홍동,Seohong-dong,2013,"POLYGON ((126.53148 33.35826, 126.53682 33.346..."
...,...,...,...,...,...
3477,1101057,무악동,Muak-dong,2013,"POLYGON ((126.96491 37.57665, 126.96494 37.573..."
3478,1101056,평창동,Pyeongchang-dong,2013,"POLYGON ((126.97389 37.6295, 126.97718 37.6286..."
3479,1101055,부암동,Buam-dong,2013,"POLYGON ((126.95522 37.60281, 126.97795 37.593..."
3480,1101054,삼청동,Samcheong-dong,2013,"POLYGON ((126.97795 37.59377, 126.98479 37.592..."


In [1]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# 1. CSV 불러오기 (위도, 경도 컬럼명 수정)
car_park = r'E:\Traffic-Safety\위치 데이터\data\불법주정차 신고현황(23년11월1일_24년3월13일).csv' 
df = pd.read_csv(car_park)
df = df.dropna(subset=["위도", "경도"])

# 위경도로 Point geometry 생성
geometry = [Point(xy) for xy in zip(df["경도"], df["위도"])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# 2. 서울시 행정동 GeoJSON 불러오기
seoul_gjson_path = r"E:\Traffic-Safety\위치 데이터\서울특별시.json"  # 경로 수정
seoul_gdf = gpd.read_file(seoul_gjson_path)

# 3. '동'으로 끝나는 이름만 필터링 (서울특별시 동만)
seoul_dong_gdf = seoul_gdf[seoul_gdf['name'].str.endswith('동')].copy()

# 4. Google 검색을 통해 서울특별시 동인지 확인하는 함수
def is_seoul_dong(dong_name):
    # 크롬 드라이버 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 창을 띄우지 않음
    driver = webdriver.Chrome(options=options)
    
    try:
        driver.get("https://www.google.com")
        search_box = driver.find_element(By.NAME, 'q')
        search_box.send_keys(f"{dong_name} 서울")  # 동 이름을 구글에 입력
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)  # 검색 결과가 로딩될 때까지 대기

        # 구글 검색 결과에서 서울특별시 관련 정보를 확인
        results = driver.find_elements(By.XPATH, "//div[@class='BVG0Nb']")
        for result in results:
            if "서울특별시" in result.text:
                return True  # 서울특별시인 경우
        return False  # 서울특별시가 아닌 경우
    finally:
        driver.quit()

# 5. '동' 기준 Spatial Join 및 필터링
valid_dongs = []
for dong in seoul_dong_gdf['name']:
    if is_seoul_dong(dong):  # 동 이름이 서울특별시인 경우만
        valid_dongs.append(dong)

# 유효한 '동'만 필터링하여 새로운 GeoDataFrame 생성
seoul_valid_dong_gdf = seoul_dong_gdf[seoul_dong_gdf['name'].isin(valid_dongs)]

# 6. 좌표 기반 Spatial Join (각 데이터 포인트가 어느 동에 속하는지 확인)
joined = gpd.sjoin(gdf, seoul_valid_dong_gdf, how="inner", predicate='within')

# 7. '동' 기준으로 count 집계
joined_grouped = joined.groupby('name').size().reset_index(name='count')

# 8. Choropleth Map 생성 (서울시 '동'별 데이터 밀도 시각화)
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11, max_bounds=True)

# '동'별 count 값을 Folium Choropleth에 적용
folium.Choropleth(
    geo_data=seoul_valid_dong_gdf,             # '동'만 포함된 GeoDataFrame
    data=joined_grouped,                       # 동별 집계된 데이터
    columns=['name', 'count'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='서울시 행정동별 데이터 밀도',
).add_to(m)

# 9. 결과를 HTML로 저장
m.save("seoul_valid_dong_density_map.html")

print("Choropleth 지도 파일이 생성되었습니다: seoul_valid_dong_density_map.html")


Choropleth 지도 파일이 생성되었습니다: seoul_valid_dong_density_map.html
